In [1]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['STT 교정', '한국어 전사']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
script_set = set()


#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            script_id = line['script']['scriptId']
            scriptITN = line['script']['scriptITN']
            if scriptITN in script_set:
                continue
            script_set.add(scriptITN)
            scriptTN = re.sub(r'[|]|[^가-힣 ]','',line['script']['scriptTN'])
            #### data preprocess end 
            
            
            #### STT 교정
            ### 'output' is list of output sorted by relevance (1st is most relevant)
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = scriptTN
            data['output'] = scriptITN
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### STT 교정 end
            
            
            #### 한국어 전사
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = scriptITN
            data['output'] = scriptTN
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 한국어 전사 end
            
            
            
            # break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 21/21 [00:03<00:00,  5.29it/s]
